In [114]:
import warnings
warnings.filterwarnings('ignore')

In [86]:
from sklearn.ensemble import RandomForestClassifier ,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


In [87]:
X,y=make_moons(n_samples=500,noise=0.3,random_state=42)
x_train,x_test,y_train,y_test=train_test_split(X,y,random_state=42)
print(x_train.shape,x_test.shape)

(375, 2) (125, 2)


### Votinng is one of  the ensemble methods here we  use it and by deafualt  it use the hard voting 

In [88]:
voting_clf=VotingClassifier(estimators=[('lr',LogisticRegression(random_state=42)),
                                        ('rf',RandomForestClassifier(random_state=42)),
                                        ('svc',SVC(random_state=42))])
voting_clf.fit(x_train,y_train)

,estimators,"[('lr', ...), ('rf', ...), ...]"
,voting,'hard'
,weights,None
,n_jobs,None
,flatten_transform,True
,verbose,False
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True


In [89]:
for name, clf in voting_clf.named_estimators_.items():
 print(name, "=", clf.score(x_test, y_test))

lr = 0.864
rf = 0.896
svc = 0.896


In [90]:
## seeing what each estimator predict
[clf.predict(x_test[:1]) for clf in voting_clf.estimators_]

[array([1]), array([1]), array([0])]

In [91]:
# the result is  by majority
voting_clf.predict(x_test[:1])


array([1])

In [92]:
voting_clf.score(x_test, y_test)

0.912

### here we will use the soft voting

In [93]:
voting_clf.voting = "soft"
# it must be seelcted because svcv has no probabilty function
voting_clf.named_estimators["svc"].probability = True
voting_clf.fit(x_train, y_train)
voting_clf.score(x_test, y_test)

0.92

## Bagging and Pasting 
#### (result is by voting or averaging)
#### bagging =bootstrap(select ranndom examples with replacement) + aggregating(aggregate results to decide)
#### pasting =bagging (without replacement(تكرار الامثلة))

| Parameter          | Meaning                                                                |
| ------------------ | ---------------------------------------------------------------------- |
| `n_estimators=500` | Train **500** decision trees.                                          |
| `max_samples=100`  | Each tree is trained on **100 samples** randomly chosen from training. |
| `bootstrap=True`   | Use **bagging** → random sampling **with replacement**.                |
| `bootstrap=False`  | Use **pasting** → random sampling **without replacement**.             |
| `n_jobs=-1`        | Use **all CPU cores** to speed up training and prediction.             |


#### normal bagging

In [94]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bg1=BaggingClassifier(DecisionTreeClassifier(),n_estimators=500,max_samples=100,n_jobs=-1,random_state=42)
bg1.fit(x_train,y_train)



,estimator,DecisionTreeClassifier()
,n_estimators,500
,max_samples,100
,max_features,1.0
,bootstrap,True
,bootstrap_features,False
,oob_score,False
,warm_start,False
,n_jobs,-1
,random_state,42
,verbose,0


#### bagginng with oob (out of bagg) validation set 

In [95]:
bg2=BaggingClassifier(DecisionTreeClassifier(),n_estimators=500,max_samples=100,oob_score=True,n_jobs=-1,random_state=42)
bg2.fit(x_train,y_train)


,estimator,DecisionTreeClassifier()
,n_estimators,500
,max_samples,100
,max_features,1.0
,bootstrap,True
,bootstrap_features,False
,oob_score,True
,warm_start,False
,n_jobs,-1
,random_state,42
,verbose,0


In [96]:
bg2.oob_score_

0.9253333333333333

In [97]:
from sklearn.metrics import accuracy_score
yPreds_bg2=bg2.predict(x_test)
accuracy_score(yPreds_bg2,y_test)

0.904

In [98]:
bg2.oob_decision_function_[:3] #probability of the first three instances 

array([[0.35579515, 0.64420485],
       [0.43513514, 0.56486486],
       [1.        , 0.        ]])

# RandomForest
## ranndom forest = bagging + decision tree model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf=RandomForestClassifier(n_estimators=500,max_leaf_nodes=100,n_jobs=-1,random_state=42)
rf_clf.fit(x_train,y_train)

,n_estimators,500
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,100
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [100]:
yPreds_rf=rf_clf.predict(x_test)
accuracy_score(yPreds_rf,y_test)

0.888

## feature impoertance 

In [101]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(iris.data, iris.target)
for score, name in zip(rnd_clf.feature_importances_, iris.data.columns):
    print(round(score, 2), name)

0.11 sepal length (cm)
0.02 sepal width (cm)
0.44 petal length (cm)
0.42 petal width (cm)


# Boosting
### sequential models (adaboost : by weight examples) OR (gradientBoost: by residuals errors)

# AdaBoost (if we select dt it must be maxdepth =1)


In [102]:
from  sklearn.ensemble import AdaBoostClassifier
adabst=AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),learning_rate=0.5,n_estimators=30,random_state=42)
adabst.fit(x_train,y_train)

,estimator,DecisionTreeC...r(max_depth=1)
,n_estimators,30
,learning_rate,0.5
,algorithm,'deprecated'
,random_state,42
,criterion,'gini'
,splitter,'best'
,max_depth,1
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0


In [103]:
yPreds_ada=adabst.predict(x_test)
print(accuracy_score(yPreds_ada,y_test))
print(accuracy_score(adabst.predict(x_train),y_train))

0.88
0.9013333333333333


# Gradient Boost (always use decisionon tree)

In [104]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(
    max_depth=2,       # Shallow trees (weak learners)
    n_estimators=3,    # Number of trees (steps)
    learning_rate=1.0, # Step size
    random_state=42
)

gbrt.fit(X, y)


,loss,'squared_error'
,learning_rate,1.0
,n_estimators,3
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,2
,min_impurity_decrease,0.0
,init,None


### GradientBoost with early stopping 

In [ ]:
gbrt_best = GradientBoostingRegressor(
max_depth=2,
learning_rate=0.05,
n_estimators=500,
n_iter_no_change=10,  #this provide stopping when no improve afte 10 iterations
random_state=42)
gbrt_best.fit(X, y)

,loss,'squared_error'
,learning_rate,0.05
,n_estimators,500
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,2
,min_impurity_decrease,0.0
,init,None


In [108]:
gbrt_best.n_estimators_

146

## HistGradientBoost

In [109]:
import pandas as pd
housing_data=pd.read_csv('housing.csv')
housing_data.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [110]:
housing=housing_data.drop('median_house_value',axis=1)
housing_labels=housing_data['median_house_value']

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder
hgb_reg = make_pipeline
(
make_column_transformer((OrdinalEncoder(), ["ocean_proximity"]),
remainder="passthrough"),  ## Leave all other columns unchanged (e.g., numeric features like median_income, etc.)
HistGradientBoostingRegressor(categorical_features=[0], random_state=42)
)
hgb_reg.fit(housing, housing_labels)


"""
categorical_features=[0]
After the transformer, "ocean_proximity" (originally column X) becomes the first column.
So we tell HGB that column 0 is a categorical feature and it will handle it specially (e.g., smarter splits).

Because just encoding a feature as integers doesn't make it categorical in the model's eyes — we must explicitly tell the model that 
it's categorical so it uses the correct logic during training.
"""

,steps,"[('columntransformer', ...), ('histgradientboostingregressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('ordinalencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


## lightgbm (enhanced gsoost like xgboost and catboosy)

In [115]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
iris = load_iris(as_frame=True)
X = iris.data
y = iris.target

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the LightGBM model
model = LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

# Fit the model
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000436 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 86
[LightGBM] [Info] Number of data points in the train set: 105, number of used features: 4
[LightGBM] [Info] Start training from score -1.219973
[LightGBM] [Info] Start training from score -1.043042
[LightGBM] [Info] Start training from score -1.043042
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

# Stacking

In [112]:

from sklearn.ensemble import StackingClassifier
stacking_clf = StackingClassifier(
estimators=
[   ##### base models ####
('lr', LogisticRegression(random_state=42)),
('rf', RandomForestClassifier(random_state=42)),
('svc', SVC(probability=True, random_state=42))
],
final_estimator=RandomForestClassifier(random_state=43), ### blender or metamodel
cv=5  # number of cross-validation folds
)
stacking_clf.fit(X_train, y_train)


,estimators,"[('lr', ...), ('rf', ...), ...]"
,final_estimator,RandomForestC...ndom_state=43)
,cv,5
,stack_method,'auto'
,n_jobs,None
,passthrough,False
,verbose,0
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
